In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


In [2]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 6A27-F1CE

 Directory of C:\Users\chi\Desktop\Earthquake

02/20/2020  11:17 PM    <DIR>          .
02/20/2020  11:17 PM    <DIR>          ..
02/20/2020  11:17 PM    <DIR>          .ipynb_checkpoints
02/19/2020  04:07 PM           214,837 Earthquake_data_preprocessing.ipynb
02/19/2020  09:54 PM            16,526 Model_train2.ipynb
02/19/2020  08:45 PM            43,710 Model_training.ipynb
02/19/2020  09:38 PM            57,336 Normalize_data.ipynb
02/20/2020  11:17 PM                72 Over_sampling.ipynb
02/20/2020  11:16 PM            89,839 Param_tune.ipynb
02/20/2020  11:19 AM           863,881 pram_sub1.csv
02/20/2020  03:38 PM           863,881 pram_sub2.csv
02/19/2020  09:38 PM        23,402,101 stand_test.csv
02/19/2020  09:37 PM        71,005,969 stand_train.csv
02/19/2020  02:44 PM           777,012 submission_format.csv
02/19/2020  11:39 PM           863,881 submission3.csv
02/19/2020  08:45 PM           863,881 submi

In [3]:
df_train = pd.read_csv('train_ready.csv', index_col='building_id')
df_test = pd.read_csv('test_ready.csv', index_col='building_id')

In [4]:
df_train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,1,1,0,...,0,0,0,0,0,0,0,0,1,0
28830,8,900,2812,2,10,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0
94947,21,363,8973,2,10,5,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
590882,22,418,10694,2,10,6,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
201944,11,131,1488,3,30,8,9,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
df_test.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,plan_configuration_f,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,0,1,0,...,0,0,0,0,0,0,0,0,1,0
99355,6,141,11987,2,25,13,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
890251,22,19,10044,2,5,4,5,0,1,0,...,0,0,0,0,0,0,0,0,1,0
745817,26,39,633,1,0,19,3,0,0,0,...,0,0,0,0,0,0,0,0,1,0
421793,17,289,7970,3,15,8,7,0,1,0,...,0,0,0,0,0,0,0,0,1,0


## Train test split

In [6]:
X = df_train.drop(columns=['damage_grade'])
y = df_train['damage_grade']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## SMOTE on trainset

In [15]:
smote = SMOTE('not majority')

In [16]:
X_sm, y_sm = smote.fit_sample(X_train, y_train)

In [17]:
print('X original: ',X_train.shape)
print('y original: ',y_train.shape)
print('X oversample: ',X_sm.shape)
print('y oversample: ',y_sm.shape)

X original:  (182420, 60)
y original:  (182420,)
X oversample:  (310956, 60)
y oversample:  (310956,)


In [18]:
y_train.value_counts()

2    103652
3     61280
1     17488
Name: damage_grade, dtype: int64

In [19]:
y_sm.value_counts()

3    103652
2    103652
1    103652
Name: damage_grade, dtype: int64

## Model fitting

In [32]:
classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=3, learning_rate=0.02,max_depth=10, nfold=5, seed=101)

In [33]:
classifier.fit(X_sm, y_sm)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.02, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nfold=5, nthread=None, num_class=3, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=101, silent=None, subsample=1, verbosity=1)

In [34]:
y_pred = classifier.predict(X_test)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.49      0.64      0.55      7636
           2       0.74      0.67      0.71     44607
           3       0.63      0.68      0.66     25938

    accuracy                           0.67     78181
   macro avg       0.62      0.66      0.64     78181
weighted avg       0.68      0.67      0.67     78181



In [36]:
classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=3, learning_rate=0.05,max_depth=20, nfold=5, seed=101)

In [37]:
classifier.fit(X_sm, y_sm)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nfold=5, nthread=None, num_class=3, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=101, silent=None, subsample=1, verbosity=1)

In [39]:
train_pred = classifier.predict(X_sm)
test_pred = classifier.predict(X_test)

In [40]:
print('Trainset classification report')
print(classification_report(y_sm, train_pred))

Trainset classification report
              precision    recall  f1-score   support

           1       0.99      0.98      0.99    103652
           2       0.93      0.94      0.94    103652
           3       0.94      0.94      0.94    103652

    accuracy                           0.95    310956
   macro avg       0.95      0.95      0.95    310956
weighted avg       0.95      0.95      0.95    310956



In [41]:
print('Testset classification report')
print(classification_report(y_test, test_pred))

Testset classification report
              precision    recall  f1-score   support

           1       0.58      0.57      0.57      7636
           2       0.75      0.77      0.76     44607
           3       0.70      0.67      0.68     25938

    accuracy                           0.72     78181
   macro avg       0.68      0.67      0.67     78181
weighted avg       0.72      0.72      0.72     78181



## Prameter tuning

In [24]:
from sklearn.metrics import fbeta_score, make_scorer

fO5_scorer = make_scorer(fbeta_score, beta=2, average = 'micro')

In [25]:
classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=3, seed=101)

In [26]:
params = {
    'learning_rate': [0.05],
    'max_depth': [20],
    'gamma': [0.5]
}

In [27]:
grid = GridSearchCV(classifier, param_grid=params, scoring = fO5_scorer, cv =5)

In [28]:
grid.fit(X_sm,y_sm, eval_set = [(X_sm, y_sm), (X_test, y_test)], early_stopping_rounds=10)

[0]	validation_0-merror:0.192638	validation_1-merror:0.288932
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.188351	validation_1-merror:0.286924
[2]	validation_0-merror:0.186093	validation_1-merror:0.285453
[3]	validation_0-merror:0.184294	validation_1-merror:0.284353
[4]	validation_0-merror:0.182124	validation_1-merror:0.283624
[5]	validation_0-merror:0.180468	validation_1-merror:0.282946
[6]	validation_0-merror:0.179169	validation_1-merror:0.281833
[7]	validation_0-merror:0.177256	validation_1-merror:0.280951
[8]	validation_0-merror:0.175858	validation_1-merror:0.280324
[9]	validation_0-merror:0.174372	validation_1-merror:0.279774
[10]	validation_0-merror:0.172662	validation_1-merror:0.279237
[11]	validation_0-merror:0.171078	validation_1-merror:0.278213
[12]	validation_0-merror:0.170129	validation_1-merror:0.277958
[13]	validation_0-merror:0.1

[61]	validation_0-merror:0.133236	validation_1-merror:0.268684
[62]	validation_0-merror:0.132847	validation_1-merror:0.268505
[63]	validation_0-merror:0.132337	validation_1-merror:0.268364
[64]	validation_0-merror:0.131965	validation_1-merror:0.268134
[65]	validation_0-merror:0.131603	validation_1-merror:0.268249
[66]	validation_0-merror:0.131225	validation_1-merror:0.268083
[67]	validation_0-merror:0.130682	validation_1-merror:0.267994
[68]	validation_0-merror:0.130238	validation_1-merror:0.267994
[69]	validation_0-merror:0.129766	validation_1-merror:0.267891
[70]	validation_0-merror:0.12929	validation_1-merror:0.267994
[71]	validation_0-merror:0.128484	validation_1-merror:0.267699
[72]	validation_0-merror:0.128001	validation_1-merror:0.267827
[73]	validation_0-merror:0.127502	validation_1-merror:0.267866
[74]	validation_0-merror:0.127097	validation_1-merror:0.26775
[75]	validation_0-merror:0.126472	validation_1-merror:0.267866
[76]	validation_0-merror:0.125896	validation_1-merror:0.2

[90]	validation_0-merror:0.121215	validation_1-merror:0.267687
[91]	validation_0-merror:0.120952	validation_1-merror:0.26793
[92]	validation_0-merror:0.120628	validation_1-merror:0.267827
[93]	validation_0-merror:0.120349	validation_1-merror:0.267738
[94]	validation_0-merror:0.119976	validation_1-merror:0.267853
[95]	validation_0-merror:0.119614	validation_1-merror:0.267546
[96]	validation_0-merror:0.119335	validation_1-merror:0.267674
Stopping. Best iteration:
[86]	validation_0-merror:0.122826	validation_1-merror:0.267226

[0]	validation_0-merror:0.192101	validation_1-merror:0.288113
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.187584	validation_1-merror:0.285415
[2]	validation_0-merror:0.184947	validation_1-merror:0.283637
[3]	validation_0-merror:0.182908	validation_1-merror:0.282537
[4]	validation_0-merror:0.181274	validation_1-merror:0.2811

[44]	validation_0-merror:0.142748	validation_1-merror:0.268838
[45]	validation_0-merror:0.141942	validation_1-merror:0.268544
[46]	validation_0-merror:0.141169	validation_1-merror:0.268569
[47]	validation_0-merror:0.14067	validation_1-merror:0.268428
[48]	validation_0-merror:0.140007	validation_1-merror:0.268582
[49]	validation_0-merror:0.139628	validation_1-merror:0.268531
[50]	validation_0-merror:0.13925	validation_1-merror:0.268505
[51]	validation_0-merror:0.138768	validation_1-merror:0.268556
[52]	validation_0-merror:0.138258	validation_1-merror:0.268326
[53]	validation_0-merror:0.137644	validation_1-merror:0.268224
[54]	validation_0-merror:0.137178	validation_1-merror:0.267814
[55]	validation_0-merror:0.136608	validation_1-merror:0.267699
[56]	validation_0-merror:0.136098	validation_1-merror:0.267699
[57]	validation_0-merror:0.135632	validation_1-merror:0.267571
[58]	validation_0-merror:0.135029	validation_1-merror:0.267738
[59]	validation_0-merror:0.134536	validation_1-merror:0.2

[72]	validation_0-merror:0.10216	validation_1-merror:0.264681
[73]	validation_0-merror:0.101568	validation_1-merror:0.264937
[74]	validation_0-merror:0.101184	validation_1-merror:0.264924
[75]	validation_0-merror:0.100663	validation_1-merror:0.264834
[76]	validation_0-merror:0.100258	validation_1-merror:0.264757
[77]	validation_0-merror:0.099742	validation_1-merror:0.264514
[78]	validation_0-merror:0.099254	validation_1-merror:0.264566
[79]	validation_0-merror:0.098761	validation_1-merror:0.264489
[80]	validation_0-merror:0.098312	validation_1-merror:0.264514
[81]	validation_0-merror:0.097879	validation_1-merror:0.264399
[82]	validation_0-merror:0.097265	validation_1-merror:0.264514
[83]	validation_0-merror:0.09681	validation_1-merror:0.264399
[84]	validation_0-merror:0.096283	validation_1-merror:0.264297
[85]	validation_0-merror:0.095795	validation_1-merror:0.264156
[86]	validation_0-merror:0.095324	validation_1-merror:0.264207
[87]	validation_0-merror:0.094995	validation_1-merror:0.2

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, num_class=3,
                                     objective='multi:softmax', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=101, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'gamma': [0.5], 'learning_rate': [0.05],
                         'max_depth': [20]},
             pre_di

In [29]:
train_pred = grid.predict(X_sm)
test_pred = grid.predict(X_test)

In [30]:
print('Trainset classification report')
print(classification_report(y_sm, train_pred))

Trainset classification report
              precision    recall  f1-score   support

           1       0.95      0.85      0.90     17488
           2       0.89      0.95      0.92    103652
           3       0.92      0.84      0.88     61280

    accuracy                           0.91    182420
   macro avg       0.92      0.88      0.90    182420
weighted avg       0.91      0.91      0.91    182420



In [31]:
print('Testset classification report')
print(classification_report(y_test, test_pred))

Testset classification report
              precision    recall  f1-score   support

           1       0.66      0.50      0.57      7636
           2       0.74      0.84      0.79     44607
           3       0.74      0.63      0.68     25938

    accuracy                           0.74     78181
   macro avg       0.72      0.66      0.68     78181
weighted avg       0.73      0.74      0.73     78181

